# Model Registry Sampler

Explores the MLflow Model Registry API.
Works as both a Databricks and Jupyter notebook.

**Overview**
* Requires MLflow 1.9.0. API changes from MLflow 1.8.0
* Starts clean: deletes all runs and the registered model.
* Trains a model 5 times with different hyperparameters.
* Gets the best model run and registers it as `production`.
* Gets the three next best model runs and registers them as `staging`.
* Loads the production model and runs predictions - using new `models` URI
* Loads the staging model and runs predictions - - using new `models` URI.

**Databricks Issues**
* If we call update_model_version() immediately after create_model_version() without a sleep, the version may not be in `READY` state
  * ERROR: INVALID_STATE: Model version andre_sklearn_registry_test version 1 has invalid status PENDING_REGISTRATION. Expected status is READY.
* This issue do not occur when running open source MLflow with Jupyter.

**Github**
* https://github.com/amesar/mlflow-examples/blob/master/model_registry/Model_Registry_Sampler.html

Last updated: 2020-06-20

### Setup

In [ ]:
import mlflow
import time
print("MLflow Version:",mlflow.__version__)
mlflow.tracking.get_tracking_uri()

In [ ]:
import os
is_databricks  = os.environ.get('DATABRICKS_RUNTIME_VERSION') is not None
is_databricks

In [ ]:
if is_databricks:
    client = mlflow.tracking.MlflowClient()
    dbutils.widgets.text("Nap Time", "2") 
    naptime = int(dbutils.widgets.get("Nap Time"))
    data_path = "/dbfs/tmp/mlflow/wine-quality.csv"
    model_name = "andre_sklearn_registry_sampler"
    experiment_name = dbutils.notebook.entry_point.getDbutils().notebook().getContext().notebookPath().get()
    print("experiment_name:",experiment_name)
else:
    data_path = "../../data/train/wine-quality-white.csv"
    mlflow.set_tracking_uri("http://localhost:5000")
    client = mlflow.tracking.MlflowClient()
    naptime = 0
    model_name = "sklearn_registry_sampler"
    experiment_name = "sklearn_registry_sampler"
    mlflow.set_experiment(experiment_name)
experiment_id = client.get_experiment_by_name(experiment_name).experiment_id
experiment_id, experiment_name, naptime

In [ ]:
if is_databricks:
    host_name = dbutils.notebook.entry_point.getDbutils().notebook().getContext().tags().get("browserHostName").get()
    uri = "https://{}/#mlflow/experiments/{}".format(host_name,experiment_id)
    displayHTML("""<b>Experiment URI:</b> <a href="{}">{}</a>""".format(uri,uri))

In [ ]:
if is_databricks:
    uri = "https://{}/#mlflow/models/{}".format(host_name,model_name)
    displayHTML("""<b>Registered Model URI:</b> <a href="{}">{}</a>""".format(uri,uri))

In [ ]:
runs = client.list_run_infos(experiment_id)
print("#runs:",len(runs))
for info in runs:
    client.delete_run(info.run_id)

### Define training

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
import mlflow
import mlflow.sklearn

In [ ]:
data = pd.read_csv(data_path)
train, test = train_test_split(data)
train_x = train.drop(["quality"], axis=1)
test_x = test.drop(["quality"], axis=1)
train_y = train[["quality"]]
test_y = test[["quality"]]

In [ ]:
def train(max_depth):
    with mlflow.start_run(run_name="reg_test") as run:
        run_id = run.info.run_uuid
        dt = DecisionTreeRegressor(max_depth=max_depth)
        dt.fit(train_x, train_y)
        predictions = dt.predict(test_x)
        mlflow.log_param("max_depth", max_depth)
        rmse = np.sqrt(mean_squared_error(test_y, predictions))
        mlflow.log_metric("rmse", rmse)
        print(f"{experiment_id} {run_id} {round(rmse,3)}")
        mlflow.sklearn.log_model(dt, "sklearn-model")

### Create runs

In [ ]:
max_depths = [1,2,4,5,16]
for x in max_depths:
    train(x)

### Create model versions

In [ ]:
runs = client.search_runs(experiment_id,"", order_by=["metrics.rmse asc"])
for run in runs:
    print(f"{round(run.data.metrics['rmse'],3)} {run.data.params}")

In [ ]:
prod_run = runs[:1][0]
round(prod_run.data.metrics['rmse'],3)

In [ ]:
staging_runs = runs[1:4]
none_runs = runs[4:]
len(staging_runs),len(none_runs)

In [ ]:
for run in staging_runs:
    print(f"{round(run.data.metrics['rmse'],3)} {run.data.params}")

In [ ]:
for run in none_runs:
    print(f"{round(run.data.metrics['rmse'],3)} {run.data.params}")

## Registry

In [ ]:
try:
    client.delete_registered_model(model_name)
    print("Deleting model")
except Exception as e:
    print(e)

In [ ]:
from mlflow.exceptions import MlflowException, RestException
try:
    registered_model = client.get_registered_model(model_name)
    print("Found existing model")
except RestException as e:
    print("Creating new model")
    client.create_registered_model(model_name)
    registered_model = client.get_registered_model(model_name)
type(registered_model), registered_model.__dict__

### Production model

In [ ]:
prod_run.info.artifact_uri

In [ ]:
source = f"{prod_run.info.artifact_uri}/sklearn-model"
client.create_model_version(model_name, source, prod_run.info.run_id)
time.sleep(naptime)

In [ ]:
versions = client.get_latest_versions(model_name)
len(versions),versions

In [ ]:
registered_model = client.get_registered_model(model_name)
versions = registered_model.latest_versions
len(versions),versions

In [ ]:
versionDetails = client.get_model_version(model_name,1)
versionDetails.__dict__

NOTE: Without above sleep, we get this error:

RestException: INVALID_STATE: Model version andre_sklearn_registry_test version 1 has invalid status PENDING_REGISTRATION. Expected status is READY.

In [ ]:
#client.update_model_version(model_name, 1, stage="Production", description="My prod version") # 1.8.0
client.transition_model_version_stage (model_name, 1, "Production") # 1.9.0

In [ ]:
client.get_model_version_stages(model_name,1)

In [ ]:
client.get_latest_versions(model_name)

In [ ]:
versionDetails = client.get_model_version(model_name,1)
versionDetails.__dict__

### Staging

In [ ]:
def show_versions(versions):
    for v in versions:
        print(f"{v.run_id} {v.version} {v.current_stage} '{v.description}'")

In [ ]:
 for j,run in enumerate(staging_runs):
    print(f"==== {j}")
    source = f"{run.info.artifact_uri}/sklearn-model"
    version = client.create_model_version(model_name, source, run.info.run_id)
    #print(version.__dict__)
    time.sleep(naptime)
    versionDetails = client.get_model_version(model_name,version.version)
    print(versionDetails.__dict__)
    #client.update_model_version(model_name, version.version, stage="Staging", description=f"My staging version {j}") # 1.8.0
    client.transition_model_version_stage(model_name, version.version, "Staging")

### Manipulate Versions

#### Update Version

In [ ]:
show_versions(client.search_model_versions(f"name='{model_name}'"))

In [ ]:
#client.update_model_version(model_name, 3, stage='None') # 1.8.0
client.transition_model_version_stage (model_name, 3, "None") # 1.9.0

In [ ]:
show_versions(client.search_model_versions(f"name='{model_name}'"))

#### Delete Version

In [ ]:
client.delete_model_version(model_name, 3)

In [ ]:
show_versions(client.search_model_versions(f"name='{model_name}'"))

### Execute version methods

In [ ]:
versions = client.search_model_versions(f"name='{model_name}'")
show_versions(versions)

In [ ]:
versions =  client.get_latest_versions(model_name)
show_versions(versions)

In [ ]:
registered_model = client.get_registered_model(model_name)
registered_model.__dict__

In [ ]:
show_versions(registered_model.latest_versions)

In [ ]:
versions = client.get_latest_versions(model_name, stages=["Production"])
show_versions(versions)

In [ ]:
versions = client.get_latest_versions(model_name, stages=["Staging"])
show_versions(versions)

### Get Model and predict

#### Production model

In [ ]:
 data_predict = data.drop(['quality'], axis=1)

In [ ]:
model_uri = f"models:/{model_name}/production"
model_uri

In [ ]:
model = mlflow.sklearn.load_model(model_uri)
model

In [ ]:
predictions = model.predict(data_predict)
pd.DataFrame(predictions).head(5)

#### Staging model

NOTE: Since there are two staging models, apparently the latest one is returned.
This is not documented.

In [ ]:
model = mlflow.sklearn.load_model(f"models:/{model_name}/staging")
model

In [ ]:
predictions = model.predict(data_predict)
pd.DataFrame(predictions).head(5)

### List Registry

In [ ]:
def dump(x,indent="  "):
  print("Registered Model")
  for k,v in x.__dict__.items():
    if k == "_latest_version":
      print("  ",k)
      for e in v:
        print("   ModelVersionDetailed:")
        for k2,v2 in e.__dict__.items():
          print("      ",k2,"-",v2)
    else:
      print("  ",k,"-",v)

In [ ]:
def list_registry():
  client = mlflow.tracking.MlflowClient()
  lst = client.list_registered_models()
  print("#register_models:",len(lst))
  for e in lst:
    dump(e)
  print("#register_models:",len(lst))

In [ ]:
list_registry()